In [4]:
pip install PyPDF2 nltk spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 6.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.4/875.4 kB 6.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 6.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.5/374.5 kB 6.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 6.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: typing_extensions
    Found exi

In [5]:
import PyPDF2
import nltk
import spacy

# Download NLTK data if needed
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/felixrosenbergernew/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [20]:
def extract_text_from_pdf(pdf_path):
    """
    This function extracts text from a pdf doc into a single stream of text.
    """
    text = ""
    with open(pdf_path, "rb") as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text


In [7]:
def preprocess_text(text):
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    # Remove non-alphanumeric tokens and lowercase
    clean_tokens = [token.lower() for token in tokens if token.isalnum()]
    return clean_tokens


In [15]:
pdf_reader = PyPDF2.PdfReader("SNOW_Q224.pdf")
print(pdf_reader)

In [22]:
SNOW = extract_text_from_pdf("SNOW_Q224.pdf")

In [23]:
preprocess_text(SNOW)

['corrected',
 'transcript',
 'total',
 'pages',
 '22',
 'copyright',
 '2001',
 'factset',
 'callstreet',
 'llc',
 'snowflake',
 'snow',
 'q2',
 '2024',
 'earnings',
 'call',
 'snowflake',
 'snow',
 'q2',
 '2024',
 'earnings',
 'call',
 'corrected',
 'transcript',
 '2',
 'copyright',
 '2001',
 'factset',
 'callstreet',
 'llc',
 'corporate',
 'participants',
 'jimmy',
 'sexton',
 'finance',
 'director',
 'head',
 'relations',
 'snowflake',
 'frank',
 'slootman',
 'chairman',
 'chief',
 'executive',
 'officer',
 'snowflake',
 'michael',
 'scarpelli',
 'chief',
 'financial',
 'officer',
 'snowflake',
 'christian',
 'kleinerman',
 'senior',
 'vice',
 'president',
 'snowflake',
 'other',
 'participants',
 'keith',
 'weiss',
 'analyst',
 'morgan',
 'stanley',
 'llc',
 'mark',
 'murphy',
 'analyst',
 'jpmorgan',
 'securities',
 'llc',
 'kirk',
 'materne',
 'analyst',
 'everco',
 're',
 'isi',
 'raimo',
 'lenschow',
 'analyst',
 'barclays',
 'capital',
 'karl',
 'keirstead',
 'analyst',
 'ubs'